In [21]:
import numpy as np
import os
from pandas import read_csv

from keras.models import Model, Sequential
from keras.layers import Dense, Input
from keras.layers import Dropout 
from keras.callbacks import ModelCheckpoint 
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import model_from_json
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.merge import add, concatenate

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.per_process_gpu_memory_fraction=0.3
session = tf.Session(config=config)

In [5]:
from config import csv_config as config
hdf5_path = config.HDF5_PATH
model_path = config.MODEL_PATH
weight_path = config.WEIGHT_PATH

In [6]:
seed = 7
np.random.seed(seed)

In [7]:
from helpers import CSVDatasetReader
reader = CSVDatasetReader(hdf5_path)
(Data, Labels) = reader.load()

In [25]:
def create_model(optimizer):
    model = Sequential()
    model.add(Dense(8, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(4, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [26]:
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

In [27]:
optimizer = ['RMSprop', 'Adam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=0)
grid_result = grid.fit(Data, Labels)

In [28]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
means = grid_result.cv_results_["mean_test_score"] 
stds = grid_result.cv_results_["std_test_score"] 
params = grid_result.cv_results_["params"] 
for mean, stdev, param in zip(means, stds, params): 
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.759115 using {'optimizer': 'Adam'}
0.755208 (0.021236) with: {'optimizer': 'RMSprop'}
0.759115 (0.021236) with: {'optimizer': 'Adam'}


In [17]:
reader.close()